In [112]:
import requests,json, time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import ast

![](http://www.listchallenges.com/f/lists/d7aacdae-74bd-42ff-b397-b73905b5867b.jpg)

## Introduction

Our goal is to scape as many New York Times articles from their api as possible. The web scraper below is desgined to open a mongo client, where all the downloaded articles will be stored and make api calls. This web scraper will run indefinately until the user termintes the code's calls. 

## New York Times API Calls

In [26]:
def call_api(url, payload, p=0):
    # Add the 'page' parameter to the payload.
    payload['page'] = p

    # Get the requested url. Error handling for bad requests should be done in
    # the calling function.
    return requests.get(url, params=payload)


def get_response(r):
    # Use json.loads to read the response text
    raw = json.loads(r.text)

    # Return the meta (hits, etc.) and docs (containing urls'n'stuff) back
    return raw['response']['meta'], raw['response']['docs']


def get_body_text(docs):

    # Grab the url from each document, if it exists, then scrape each url for
    # its body text. If we get any errors along the way, continue on to the
    # next document / url to be scraped.
    result = []
    for d in docs:

        # Make a copy of the doc's dictionary
        doc = d.copy()

        # If there's no url (not sure why this happens sometimes) then ditch it
        if not doc['web_url']:
            continue

        # Scrape the doc's url, return a soup object with the url's text.
        soup = get_soup(doc['web_url'])
        if not soup:
            continue

        # Find all of the paragraphs with the correct class.
        # This class tag is specific to NYT articles.
        body = soup.find_all('p', class_= "story-body-text story-content")
        if not body:
            continue

        # Join the resulting body paragraphs' text (returned in a list).
        doc['body'] = '\n'.join([x.get_text() for x in body])

        print (doc['web_url'])
        result.append(doc)

    return result

def get_soup(url):
    # Header to be passed in to NYT when scraping article text.
    agent  = 'DataWrangling/1.1 (http://zipfianacademy.com; '
    agent += 'class@zipfianacademy.com)'
    headers = {'user_agent': agent}

    # Wrap in a try-except to prevent a maxTry connection error from erroring
    # out the program. Return None if there are any issues.
    try:
        r = requests.get(url, headers=headers)
    except:
        return None

    # Just in case there was a normal error returned. Pass back None.
    if r.status_code != 200: return None

    # Otherwise return a soupified object containing the url text encoded in
    # utf-8. Will toss back errors on some pages without the encoding in place.
    return BeautifulSoup(r.text.encode('utf-8'))

In [27]:
# Url for NYT dev api
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
api_key_path = "/Users/databeast03/.secret_keys/nyt_api_key.txt"
with open(api_key_path, 'r') as handle:
    API_KEY = handle.read()

In [40]:
# Request all of the newest articles matching the search term
#             "section_name": ['U.S.', 'World', 'Business Day'],
payload  = {'sort': 'newest',
            'api-key': API_KEY}

In [41]:
pages = [1,2]
docs_list = []
for page in pages:
# Call the API with BaseURL + params and page
    r = call_api(NYT_URL, payload, page)
    meta, docs = get_response(r)
    docs_body = get_body_text(docs)
    docs_list.append(docs_body)

In [95]:
docs[3]["web_url"]

'https://www.nytimes.com/aponline/2018/09/25/us/ap-us-animal-shelter-pepper-spray.html'

In [117]:
soup = BeautifulSoup(r.text.encode('utf-8'), 'html.parser')

In [ ]:
def scarp_articles_text(articles_df):
    
    # Unable false positive warning from Pandas dataframe manipulation
    pd.options.mode.chained_assignment = None
    
    articles_df['article_text'] = 'NaN'
    session = requests.Session()
    
    print('Scarping articles body text...'),
    
    for j in range(0, len(articles_df)):
        
        url = articles_df['url'][j]
        req = session.get(url)
        soup = BeautifulSoup(req.text, 'lxml')

        # Get only HTLM tags with article content
        # Articles through 1986 are found under different p tag 
        paragraph_tags = soup.find_all('p', class_= 'story-body-text story-content')
        if paragraph_tags == []:
            paragraph_tags = soup.find_all('p', itemprop = 'articleBody')

        # Put together all text from HTML p tags
        article = ''
        for p in paragraph_tags:
            article = article + ' ' + p.get_text()

        # Clean article replacing unicode characters
        article = article.replace(u'\u2018', u"'").replace(u'\u2019', u"'").replace(u'\u201c', u'"').replace(u'\u201d', u'"')

        # Copy article's content to the dataframe
        articles_df['article_text'][j] = article
    
    print('Done!')
    
    return articles_df

In [78]:
session = requests.Session()

In [80]:
req = session.get(docs[2]["web_url"])

In [82]:
soup = BeautifulSoup(req.text, 'lxml')

In [84]:
paragraph_tags = soup.find_all('p', class_= 'story-body-text story-content')

In [85]:
paragraph_tags

[]

In [86]:
paragraph_tags = soup.find_all('p', itemprop = 'articleBody')

In [87]:
paragraph_tags

[]